### Aplicação do algoritmo Apriori Para Minerar Regras de Associação em Titulos de Artigos 

### Aluno:  Leandro Rossetti

#### O proposito desse trabalho e demostrar a aplicação de Regras de Associção em um conjunto de dados contendo referencias bibliográficas de artigos baixadas da internet a fim de obter insigts uteis na filtragfem de textos para revisões sistematizadas de literatura.

#### Regras de associação geralmente são utilizadas em sistemas de recomendação e frequentemente estão associadas ao contexto de um carrinho de compras.

#### Nesse trabalho em utilizei a convenção de tratar cada artigo como se fosse uma transação e cada palavra d otitulo como se fosse um item em um carrinho de compras, ao qual foi atribuido um numero de transão e uma quantidade ficticios, a fim depermitir a aplicação do algoritmo Apriori 




### Importção das bibliotecas usadas ao longo do código

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules,fpgrowth,fpmax 
import nltk
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import bibtexparser
from bibtexparser.bparser import BibTexParser
from bibtexparser.customization import homogenize_latex_encoding
import sys


%matplotlib inline

### Crio algumas variaveis que serão utilizadas ao longo da aplicação

#### stop_words --> Utilizo a biblioteca  nltk.corpus.stopwords, essa biblioteca contém um conjunto de palavras que devem ser desconsideradas no tratamento detextos, como por exemplo Artigos, Preposições, sufixos, Auxiliares, etc...


### Existem 150 arquivos com referencia sbibliograficas no formato Bibitex

#### Leio os arquivos com as referencias no formato Bibtex ".bib"e acumulo o conteudo em uma lista
#### Faço um parsing em cada um dos arquivos, 
#### Extraio apenas as informações de Titulo, Autor e Ano e acumulo em uma lista

In [124]:
listItens = []

for i in range(0,151):
    with open('bib\\{}.bib'.format(i), encoding='iso-8859-1') as bibtex_file:
        print('Processando o arquivo: {}.bib'.format(i))
        parser = BibTexParser()
        parser.customization = homogenize_latex_encoding
        bib_database = bibtexparser.load(bibtex_file, parser=parser)
      
    for i in bib_database.entries:
        if 'author' in i:
            if 'title' in i:
                if 'year' in i:
                    listItens.append([i['title'], i['author'], i['year']])  


Processando o arquivo: 0.bib
Processando o arquivo: 1.bib
Processando o arquivo: 2.bib
Processando o arquivo: 3.bib
Processando o arquivo: 4.bib
Processando o arquivo: 5.bib
Processando o arquivo: 6.bib
Processando o arquivo: 7.bib
Processando o arquivo: 8.bib
Processando o arquivo: 9.bib
Processando o arquivo: 10.bib
Processando o arquivo: 11.bib
Processando o arquivo: 12.bib
Processando o arquivo: 13.bib
Processando o arquivo: 14.bib
Processando o arquivo: 15.bib
Processando o arquivo: 16.bib
Processando o arquivo: 17.bib
Processando o arquivo: 18.bib
Processando o arquivo: 19.bib
Processando o arquivo: 20.bib
Processando o arquivo: 21.bib
Processando o arquivo: 22.bib
Processando o arquivo: 23.bib
Processando o arquivo: 24.bib
Processando o arquivo: 25.bib
Processando o arquivo: 26.bib
Processando o arquivo: 27.bib
Processando o arquivo: 28.bib
Processando o arquivo: 29.bib
Processando o arquivo: 30.bib
Processando o arquivo: 31.bib
Processando o arquivo: 32.bib
Processando o arquiv

### Trasnformo a lista em um DataFrame para facilitar a manipulação dos dados

In [3]:
dfBibItens = pd.DataFrame(listItens)
dfBibItens.head()

,0,1,2
0,R{E}SI{M}A: {A}n {A}ssistive {P}aradigm to {S}...,"And{\~A}{^2}, Bruno and Baglio, Salvatore and ...",2014
1,{M}ultisensor - {B}ased {O}bject {D}etection i...,"Patel, Charmi T. and Mistry, Vaidehi J. and De...",2018
2,J{U}VO - {A}n {A}id for the {V}isually {I}mpaired,"Gianani, Sejal and Mehta, Abhishek and Motwani...",2018
3,{A} low cost smart shopping facilitator for vi...,"Mathankumar, M. and Sugandhi, N.",2013
4,{C}hange analysis for gait impairment quantifi...,"Salaheldin, A. and ElSayed, M. and Alsebai, A....",2010


### Existem 11 arquivos com referências  no formato .csv
#### Carrego cada um desse arquivos e acumulo em uma lista
#### Adiciono a essa lista o dataset de referencias Bibitex criado no passo anterior
#### Como existem arquivos com conteudos diferentes eu faço o tratamento de cada um para  Extrair apenas as informações de Titulo, Autor e Ano e sobreescrevoi o dataset original com esse subset da dados
#### Faço merge de todos os Dataframes em um só
#### Removo as duplicidade da lista 

In [126]:
listDf = []

# Carrego em DataFrames os arquivos contendo as referencias no formato .csv  
for i in range(1,12):
    listDf.append(pd.read_csv('csv\\{}.csv'.format(i)))

listDf.append(dfBibItens)

listDf[0] = listDf[0][['Title','Authors','Publication Year']]
listDf[1] = listDf[1][['Document Title','Authors','Publication Year']]
 
for i in range(2,11):
    listDf[i] = listDf[i][['Item Title','Authors','Publication Year']]     

listTitle = []

for i in listDf[0].iterrows():
    listTitle.append(i[1][0])

for i in range(1,11):
    for j in listDf[i].iterrows():
        listTitle.append(j[1][0])

for i in listDf[11].iterrows():
    listTitle.append(i[1][0])

qntComDuplicidades = len(listTitle) 
listTitle = list(dict.fromkeys(listTitle))
qntSemDuplicidades = len(listTitle) 

dfTitulos = pd.DataFrame(listTitle)


#### Aqui apenas apresentro umas informações do DataFrame unificado

In [127]:
print('Havian {} registros duplicados no Dataframe'.format(qntComDuplicidades-qntSemDuplicidades))
print('Existem {} registros unicos no DataFrame'.format(qntSemDuplicidades))
print(dfTitulos.head())

Havian 5217 registros duplicados no Dataframe
Existem 17490 registros unicos no DataFrame
                                0
0  Mucopolysaccharidosis Type III
1                      Presbyopia
2                       Chlamydia
3                Alström Syndrome
4                        Cataract


### Criei a função ProcessText para tratar os dados de cada titulo

#### Essa função recebe um inteiro representando o index desse artigo na lista de artigos e o titulo desse artigo, remove as stop_words desse titulo e separa as palavras restantes eu ema lista

In [112]:
stop_words = set(stopwords.words('english'))
articles = []
                
def ProcessText(text, index):
    word_tokens = word_tokenize(text)
    itens = [x for x in word_tokens if x not in stop_words]

    for i in itens:
        articles.append([index, i, 1])

#### Faço uma chamada a função ProcessText passando como parametro cada um dos titulos do Dataframe dfTitulos 

In [113]:
cont = 1

for i in dfTitulos.iterrows():
    title = i[1][0]
    if type(title) == str:
        if len(title) > 5:       
            title = re.sub("[^a-zA-Z]", " ", i[1][0])
            ProcessText(title, cont)
            cont = cont+1    


#### Crio o DataFrame "df" a partir da lista "articles"
#### Nesse novo DataFrame a coluna Article é um indice numérico e tem a função de simular um numero de transação  
#### A coluna Item recebeu os titulos dos artigos, e em um carrinho de compras ela seria o item comprado
#### A coluna Qtd representa a quantidade do item, o que nesse contexto sempre será 1

In [114]:
df = pd.DataFrame(articles)
df.columns = ['Article','Item','Qtd']

#### Printo os 5 preimeiros registros do DataFrame apenas para visualizção 

In [115]:
df.head(10)

,Article,Item,Qtd
0,1,Mucopolysaccharidosis,1
1,1,Type,1
2,1,III,1
3,2,Presbyopia,1
4,3,Chlamydia,1
5,4,Alstr,1
6,4,Syndrome,1
7,5,Cataract,1
8,6,Glaucoma,1
9,7,Visual,1


#### Faço o pivotamento do DataFrame para possibilitar a aplicação do algoritmop Apriori

In [116]:
dfPiv = df \
    .groupby(['Article', 'Item'])['Qtd'] \
    .max().unstack().reset_index().fillna(0) \
    .set_index('Article') 

#### Printo os 5 preimeiros registros do DataFrame apenas para visualizção 

In [117]:
dfPiv.head()

Item,A,AA,AAL,AB,ABAC,ABC,AC,ACCV,ACd,ACnet,...,zetimibe,zig,zigbee,zilsartan,zipper,zirconium,zone,zones,zure,zwischen
Article,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Defino o modelo que sera usado, nesse caso utilizei o modelo Apriori forneceido pela biblioteca mlxtend (http://rasbt.github.io/mlxtend/)

In [118]:
modelDf = apriori(dfPiv, min_support = 0.05, use_colnames = True)
modelDf.head()

,support,itemsets
0,0.390931,(A)
1,0.130539,(B)
2,0.231917,(C)
3,0.177883,(D)
4,0.208989,(E)


#### Ttreinando o modelo Apriori com o DataFrame  pivotedo 
#### Mapeando as regras para um objeto à parte

In [119]:
regras = association_rules(modelDf, metric ='lift', min_threshold = 1) 

#### Formatando um objeto com as regras ordenadas por 'support' e 'confidence' para exibição

In [120]:
regras = regras.sort_values(['confidence', 'lift'], ascending =[False, False])

#### Exibindo os 20 primeiros registros to com as regras ordenadas por 'support' e 'confidence' para exibição

In [121]:
regras.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
34,(ased),(B),0.050089,0.130539,0.050089,1.0,7.660534,0.043550,inf
85,(nvironment),(E),0.059294,0.208989,0.059294,1.0,4.784952,0.046903,inf
100,(ntelligent),(I),0.052147,0.255189,0.052147,1.0,3.918665,0.038840,inf
123,(mart),(S),0.080336,0.287781,0.080336,1.0,3.474866,0.057217,inf
228,"(mart, E)",(S),0.055578,0.287781,0.055578,1.0,3.474866,0.039584,inf
